In [1]:
# coding: utf-8
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# import plotly.graph_objs as go
# import plotly.tools as tls
from collections import Counter
import time
import gc
import lightgbm as lgb
import xgboost as xgb
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
warnings.filterwarnings('ignore')
np.random.seed(2018)

In [2]:
train = pd.read_table('../data/round2_train.txt', delim_whitespace=True, index_col=None)
test = pd.read_table('../data/round2_ijcai_18_test_a_20180425.txt', delim_whitespace=True, index_col=None)
test_b = pd.read_table('../glq/2.data/round2_test_b.txt', delim_whitespace=True, index_col=None)

train = train.drop_duplicates().reset_index(drop=True)
data = pd.concat([train, test, test_b]).reset_index(drop=True)
del train, test

In [3]:
timeFeat = pd.read_csv('featFile-b/timeFeat.csv')
data = pd.concat([data, timeFeat], axis=1)
del timeFeat

In [4]:
catFeat = pd.read_csv('featFile-b/catFeat.csv')
data = pd.concat([data, catFeat], axis=1)
del catFeat

In [5]:
shopFeat = pd.read_csv('featFile-b/shopFeat.csv')
data = pd.concat([data, shopFeat], axis=1)
del shopFeat

In [6]:
import gc
gc.collect()

56

# haobin's feat

In [7]:
haobin_dir = 'featFile-b/haobin/'

### simple feats

In [8]:
simple_entity_file = haobin_dir +  "simple_uesful_features_b.csv"
simple_entity_features = pd.read_csv(simple_entity_file)
data = pd.concat([data, simple_entity_features], axis=1)

In [9]:
simple_cols = simple_entity_features.columns.tolist()
simple_cols

['shop_item_sales_min',
 'cat_item_cnt',
 'shop_item_price_min',
 'shop_user_age_mean',
 'shop_cat_num']

In [10]:
del simple_entity_features

### click_buy_ts_slicewin

In [11]:
click_but_sub_dir = 'click_buy_ts_slicewin/'

In [12]:
click_buy_feats = pd.DataFrame()
# 读取 click&buy特征
entity_names = ['user_id', 'item_id', 'shop_id', 'user_id_item_id', 'user_id_shop_id']
# 5. 7号上午entity点击/购买相关特征  1/2/4
for name in entity_names:
    click_win = pd.read_pickle(haobin_dir+ click_but_sub_dir + name + "_click&buy_ts_slicewin.pkl")
    click_win.fillna(0, inplace=True)
#     if not click_buy_feats:
#         click_buy_feats = click_win
#     else:
    click_buy_feats = pd.concat([click_buy_feats, click_win], axis=1)
    del click_win
data = pd.concat([data, click_buy_feats], axis=1)

In [13]:
click_buy_cols = click_buy_feats.columns.tolist()
click_buy_cols

['buy_context_ts_user_id_hhour1_before',
 'dis_context_ts_user_id_hhour1_before',
 'buy_context_ts_user_id_hhour2_before',
 'dis_context_ts_user_id_hhour2_before',
 'buy_context_ts_user_id_hhour4_before',
 'dis_context_ts_user_id_hhour4_before',
 'dis_context_ts_user_id_hhour1_after',
 'dis_context_ts_user_id_hhour2_after',
 'dis_context_ts_user_id_hhour4_after',
 'buy_context_ts_item_id_hhour1_before',
 'dis_context_ts_item_id_hhour1_before',
 'buy_context_ts_item_id_hhour2_before',
 'dis_context_ts_item_id_hhour2_before',
 'buy_context_ts_item_id_hhour4_before',
 'dis_context_ts_item_id_hhour4_before',
 'dis_context_ts_item_id_hhour1_after',
 'dis_context_ts_item_id_hhour2_after',
 'dis_context_ts_item_id_hhour4_after',
 'buy_context_ts_shop_id_hhour1_before',
 'dis_context_ts_shop_id_hhour1_before',
 'buy_context_ts_shop_id_hhour2_before',
 'dis_context_ts_shop_id_hhour2_before',
 'buy_context_ts_shop_id_hhour4_before',
 'dis_context_ts_shop_id_hhour4_before',
 'dis_context_ts_shop_

### context_last_next_click

In [14]:
context_last_next_sub_dir = 'context_last_next_click/'

In [15]:
context_last_next_feat = pd.DataFrame()
entity_names = ['user_id', 'item_id', 'shop_id', 'user_id_item_id', 'user_id_shop_id']
col_add = []
for name in entity_names:
    context_last_click_gap = pd.read_pickle(haobin_dir + context_last_next_sub_dir  + name + '_context_last_click_time_b.pkl')
    context_last_click_gap.fillna(context_last_click_gap.max() + 1, inplace=True)
    context_last_next_feat = pd.concat([context_last_next_feat, context_last_click_gap], axis=1)

    try:
        context_next_click_gap = pd.read_pickle(haobin_dir + context_last_next_sub_dir  + name + '_context_next_click_time_b.pkl')
        context_next_click_gap.fillna(context_next_click_gap.max() + 1, inplace=True)
        context_last_next_feat = pd.concat([context_last_next_feat, context_next_click_gap], axis=1)
        del context_last_click_gap
        del context_next_click_gap
    except IOError:
        pass
data = pd.concat([data, context_last_next_feat], axis=1)

In [16]:
context_last_next_cols = context_last_next_feat.columns.tolist()
context_last_next_cols

['user_id_context_time_gap',
 'user_id_context_time_gap_next',
 'item_id_context_time_gap',
 'item_id_context_time_gap_next',
 'shop_id_context_time_gap',
 'shop_id_context_time_gap_next',
 'user_id_item_id_context_time_gap',
 'user_id_item_id_context_time_gap_next',
 'user_id_shop_id_context_time_gap']

In [17]:
del context_last_next_feat

In [18]:
gc.collect()

56

# singleCVR

In [19]:
singleCVR = pd.read_csv('featFile-b/singleCvrFeat.csv')

In [20]:
for cvr_col in singleCVR.columns.tolist():
    data[cvr_col] = singleCVR[cvr_col].tolist()

singleCVR_cols = singleCVR.columns.tolist()
singleCVR_cols

['item_id_HistoryCVR',
 'shop_id_HistoryCVR',
 'user_id_HistoryCVR',
 'user_age_level_HistoryCVR',
 'context_page_id_HistoryCVR',
 'item_brand_id_HistoryCVR',
 'item_price_level_HistoryCVR',
 'item_sales_level_HistoryCVR',
 'item_collected_level_HistoryCVR',
 'item_cat_1_HistoryCVR',
 'user_star_level_HistoryCVR',
 'shop_review_num_level_HistoryCVR',
 'shop_star_level_HistoryCVR',
 'hour_HistoryCVR',
 'hour_range_HistoryCVR']

In [21]:
del singleCVR

# clickTradeRatio

In [22]:
clickTradeRatioFeat = pd.read_csv('featFile-b/clickTradeRatioFeat.csv')

In [23]:
for ratio_col in clickTradeRatioFeat.columns.tolist():
    data[ratio_col] = clickTradeRatioFeat[ratio_col].tolist()

In [24]:
clickTradeRatio_cols = clickTradeRatioFeat.columns.tolist()
clickTradeRatio_cols

['his_item_cat_1_user_age_level_clickRatio',
 'his_item_cat_1_user_age_level_tradeRatio',
 'his_item_cat_1_user_gender_id_clickRatio',
 'his_item_cat_1_user_gender_id_tradeRatio',
 'his_item_cat_1_user_occupation_id_clickRatio',
 'his_item_cat_1_user_occupation_id_tradeRatio',
 'his_item_cat_1_user_age_level_clickRatio.1',
 'his_item_cat_1_user_age_level_tradeRatio.1',
 'his_item_cat_1_user_gender_id_clickRatio.1',
 'his_item_cat_1_user_gender_id_tradeRatio.1',
 'his_item_cat_1_user_occupation_id_clickRatio.1',
 'his_item_cat_1_user_occupation_id_tradeRatio.1',
 'his_shop_id_user_age_level_clickRatio',
 'his_shop_id_user_age_level_tradeRatio',
 'his_shop_id_user_gender_id_clickRatio',
 'his_shop_id_user_gender_id_tradeRatio',
 'his_shop_id_user_occupation_id_clickRatio',
 'his_shop_id_user_occupation_id_tradeRatio',
 'his_item_price_level_user_age_level_clickRatio',
 'his_item_price_level_user_age_level_tradeRatio',
 'his_item_price_level_user_gender_id_clickRatio',
 'his_item_price_le

In [25]:
del clickTradeRatioFeat

# <font color="red">2018.5.12添加</font>

# user当天/当小时的点击量

In [26]:
initUserClickCountFeat = pd.read_csv('featFile-b/initUserClickCountFeat.csv')
initUserClickCountFeat_cols = initUserClickCountFeat.columns.tolist()
initUserClickCountFeat_cols

['initUserTotalClickCount',
 'initUserDayClickCount',
 'initUserHourClickCount',
 'initUserTotalItemClickCount',
 'initUserDayItemClickCount',
 'initUserTotalBrandClickCount',
 'initUserDayBrandClickCount',
 'initUserTotalCat1ClickCount',
 'initUserDayCat1ClickCount',
 'initUserHourCat1ClickCount',
 'initUserTotalCa2ClickCount',
 'initUserDayCat2ClickCount',
 'initUserTotalShopClickCount',
 'initUserDayShopClickCount',
 'initUserHourShopClickCount',
 'initUserTotalPriceClickCount',
 'initUserDayPriceClickCount',
 'initUserTotalCityClickCount',
 'initUserDayCityClickCount']

In [27]:
for _col in initUserClickCountFeat_cols:
    data[_col] = initUserClickCountFeat[_col].tolist()

In [28]:
del initUserClickCountFeat

# user前一天的点击量

In [29]:
preDayUserClickCountFeat = pd.read_csv('featFile-b/preDayUserClickCountFeat.csv')
preDayUserClickCountFeat_cols = preDayUserClickCountFeat.columns.tolist()
preDayUserClickCountFeat_cols

['preDay_initUserDayClickCount',
 'preDay_initUserDayItemClickCount',
 'preDay_initUserDayBrandClickCount',
 'preDay_initUserDayCat1ClickCount',
 'preDay_initUserDayCat2ClickCount',
 'preDay_initUserDayShopClickCount',
 'preDay_initUserDayPriceClickCount',
 'preDay_initUserDayCityClickCount']

In [30]:
for _col in preDayUserClickCountFeat_cols:
    data[_col] = preDayUserClickCountFeat[_col].tolist()

In [31]:
del preDayUserClickCountFeat

# 除user之外的当天/当小时的点击特征

In [32]:
initClickCountExceptUser = pd.read_csv('featFile-b/initClickCountExceptUserFeat.csv')
initClickCountExceptUser_cols = initClickCountExceptUser.columns.tolist()
initClickCountExceptUser_cols

['initItemDayMeanClickCount',
 'initShopDayMeanClickCount',
 'initItemClickCount',
 'initItemDayClickCount',
 'initItemHourClickCount',
 'initShopTotalClickCount',
 'initShopDayClickCount',
 'initShopHourClickCount',
 'initBrandTotalClickCount',
 'initBrandDayClickCount',
 'initBrandHourClickCount',
 'initCat1TotalClickCount',
 'initCat1DayClickCount',
 'initCat1HourClickCount',
 'initCa2TotalClickCount',
 'initCa2DayClickCount']

In [33]:
for _col in initClickCountExceptUser_cols:
    data[_col] = initClickCountExceptUser[_col].tolist()

In [34]:
del initClickCountExceptUser

# 除user之外的前一天的点击量

In [35]:
preDayClickTradeCountExceptUserFeat = pd.read_csv('featFile-b/preDayClickTradeCountExceptUserFeat.csv')
preDayClickTradeCountExceptUserFeat_cols = preDayClickTradeCountExceptUserFeat.columns.tolist()
preDayClickTradeCountExceptUserFeat_cols

['Unnamed: 0',
 'preDay_initItemClickCount',
 'preDay_initItemTradeCount',
 'preDay_initShopClickCount',
 'preDay_initShopTradeCount',
 'preDay_initBrandClickCount',
 'preDay_initBrandTradeCount',
 'preDay_initCat1ClickCount',
 'preDay_initCat1TradeCount',
 'preDay_initCat2ClickCount',
 'preDay_initCat2TradeCount']

In [36]:
for _col in preDayClickTradeCountExceptUserFeat_cols:
    data[_col] = preDayClickTradeCountExceptUserFeat[_col].tolist()

In [37]:
del preDayClickTradeCountExceptUserFeat

# User当天已有的购买量

In [38]:
userDayAlreadyTradeCount = pd.read_csv('featFile-b/userDayAlreadyTradeCount.csv')
userDayAlreadyTradeCount_cols = userDayAlreadyTradeCount.columns.tolist()
userDayAlreadyTradeCount_cols

['userTodayAlreadyTradeCount',
 'userItemTodayAlreadyTradeCount',
 'userCat1TodayAlreadyTradeCount',
 'userShopTodayAlreadyTradeCount',
 'userBrandTodayAlreadyTradeCount']

In [39]:
for _col in userDayAlreadyTradeCount_cols:
    data[_col] = userDayAlreadyTradeCount[_col].tolist()

In [40]:
del userDayAlreadyTradeCount

In [41]:
gc.collect()

561

# <font color="red">2018.5.13添加</font>

# User当天的点击排序

In [42]:
userDayClickRankFeat = pd.read_csv('featFile-b/userDayClickRank.csv')
userDayClickRankFeat_cols = userDayClickRankFeat.columns.tolist()
userDayClickRankFeat_cols

['userDayClickRank',
 'userDayItemClickRank',
 'userDayCatClickRank',
 'userDayShopClickRank',
 'userDayBrandClickRank']

In [43]:
for _col in userDayClickRankFeat_cols:
    data[_col] = userDayClickRankFeat[_col].tolist()

In [44]:
del userDayClickRankFeat

# 当前item/shop/cat与平均的差值

In [45]:
horizonClickSubMeanFeat = pd.read_csv('featFile-b/horizonClickSubMeanFeat.csv')
horizonClickSubMeanFeat_cols = horizonClickSubMeanFeat.columns.tolist()
horizonClickSubMeanFeat_cols

['item_price_level_subHorizonMean',
 'item_sales_level_subHorizonMean',
 'shop_review_num_level_subHorizonMean',
 'shop_score_service_subHorizonMean',
 'shop_score_delivery_subHorizonMean',
 'shop_score_description_subHorizonMean',
 'shop_review_positive_rate_subHorizonMean']

In [46]:
for _col in horizonClickSubMeanFeat_cols:
    data[_col] = horizonClickSubMeanFeat[_col].tolist()

In [47]:
del horizonClickSubMeanFeat

# 交叉特征转化率

## age/gender 交叉特征转化率

In [48]:
crossCvrFeat_gender_age = pd.read_csv('featFile-b/crossCvrFeat-gender-age.csv')
crossCvrFeat_gender_age_cols = crossCvrFeat_gender_age.columns.tolist()
crossCvrFeat_gender_age_cols

['user_gender_id_item_price_level_CrossCVR',
 'user_gender_id_item_brand_id_CrossCVR',
 'user_gender_id_item_cat_1_CrossCVR',
 'user_gender_id_item_id_CrossCVR',
 'user_age_level_item_price_level_CrossCVR',
 'user_age_level_item_brand_id_CrossCVR',
 'user_age_level_item_cat_1_CrossCVR',
 'user_age_level_item_id_CrossCVR']

In [49]:
for _col in crossCvrFeat_gender_age_cols:
    data[_col] = crossCvrFeat_gender_age[_col].tolist()

In [50]:
del crossCvrFeat_gender_age

## occu 交叉特征转化率

In [51]:
crossCvrFeat_occu = pd.read_csv('featFile-b/crossCvrFeat-occu.csv')
crossCvrFeat_occu_cols = crossCvrFeat_occu.columns.tolist()
crossCvrFeat_occu_cols

['user_occupation_id_item_id_CrossCVR',
 'user_occupation_id_item_price_level_CrossCVR',
 'user_occupation_id_item_brand_id_CrossCVR',
 'user_occupation_id_item_cat_1_CrossCVR']

In [52]:
for _col in crossCvrFeat_occu_cols:
    data[_col] = crossCvrFeat_occu[_col].tolist()

In [53]:
del crossCvrFeat_occu

# shop 交叉特征转化率

In [54]:
crossCvrFeat_shop = pd.read_csv('featFile-b/crossCvrFeat-shop.csv')
crossCvrFeat_shop_cols = crossCvrFeat_shop.columns.tolist()
crossCvrFeat_shop_cols

['shop_id_item_id_CrossCVR',
 'shop_id_item_price_level_CrossCVR',
 'shop_id_item_brand_id_CrossCVR',
 'shop_id_item_cat_1_CrossCVR']

In [55]:
for _col in crossCvrFeat_shop_cols:
    data[_col] = crossCvrFeat_shop[_col].tolist()

In [56]:
del crossCvrFeat_shop

# glq的交叉转化率

In [57]:
glq_crossCvr = pd.read_pickle('featFile-b/ratio')
glq_crossCvr_cols = glq_crossCvr.columns.tolist()
glq_crossCvr_cols

['item_id-item_brand_id-ratio-7',
 'item_id-user_id-ratio-7',
 'item_id-shop_id-ratio-7',
 'item_brand_id-user_id-ratio-7',
 'item_brand_id-shop_id-ratio-7',
 'user_id-shop_id-ratio-7']

In [58]:
del click_buy_feats,context_last_click_gap
gc.collect()

435

In [59]:
data['index'] = data.index.tolist()
index_time = data[['index', 'context_timestamp']].sort_values(by='context_timestamp').reset_index(drop=True)

In [60]:
data.drop('index', inplace=True,axis = 1)
_t = pd.concat([index_time, glq_crossCvr.reset_index(drop=True)], axis=1).sort_values(by='index').reset_index(drop=True)[glq_crossCvr_cols]

In [61]:
for _col in glq_crossCvr_cols:
    data[_col] = _t[_col].tolist()

In [62]:
del glq_crossCvr, _t

# 冷启动特征

In [63]:
hour_browse_is_last = pd.read_pickle('../glq/2.data/myfeature/hour_browse_is_last')
hour_browse_is_last_cols = hour_browse_is_last.columns.tolist()
hour_browse_is_last_cols

['user_id-hour-browse-is-first', 'user_id-hour-browse-is-last']

In [64]:
for _col in hour_browse_is_last_cols:
    data[_col] = hour_browse_is_last[_col].tolist()

In [65]:
del hour_browse_is_last
gc.collect()

310

# userClickSubMeanUnderCat

In [77]:
# user_click_sub_mean_under_cat = pd.read_csv('featFile-b/userClickSubMeanUnderCat.csv')
# user_click_sub_mean_under_cat_cols = user_click_sub_mean_under_cat.columns.tolist()
# user_click_sub_mean_under_cat_cols

['item_price_levelsubUserCatHisMean',
 'item_sales_levelsubUserCatHisMean',
 'shop_review_num_levelsubUserCatHisMean',
 'shop_score_servicesubUserCatHisMean',
 'shop_score_deliverysubUserCatHisMean',
 'shop_score_descriptionsubUserCatHisMean',
 'shop_review_positive_ratesubUserCatHisMean']

In [78]:
# for _col in user_click_sub_mean_under_cat_cols:
#     data[_col] = user_click_sub_mean_under_cat[_col].tolist()

In [79]:
# del user_click_sub_mean_under_cat
# gc.collect()

13945

# 合并

In [80]:
just_7 = True # 只用第7填
if just_7:
    data_7 = data[data.day == 7]
print(data_7.shape)

(2806831, 254)


In [81]:
if just_7:
    train, test = data_7[data_7.is_trade.notnull()], data_7[data_7.is_trade.isnull()]
    off_train, off_valid = train[train.hour < 10], train[train.hour >= 10]
del data_7

In [82]:
# 必须drop的特征列
drop_features = ['instance_id', 'item_category_list',
                 'item_property_list', 'context_id',
                 'is_trade', 'predict_category_property',
                 'context_timestamp']

drop_features += list(filter(lambda x: '.1' in x or '_y' in x or 'Unnamed' in x, data.columns.tolist()))

drop_features += ['day', 'halfHour', 'total_hour'] # 保留minute
drop_features += ['shop_score_service_level',
 'shop_score_delivery_level',
 'shop_score_description_level',
 'shop_review_positive_rate_level'] # 170291
drop_features += list(filter(lambda x: 'clickRatio' in x, data.columns.tolist()))
drop_features += preDayUserClickCountFeat_cols # 加了下降
drop_features += initClickCountExceptUser_cols # 加了lgb稍升，xgb下降

drop_features += crossCvrFeat_gender_age_cols # 加了下降
drop_features += crossCvrFeat_occu_cols # 加了下降
drop_features += crossCvrFeat_shop_cols # 加了下降
drop_features += hour_browse_is_last_cols

# crossCvr_cols = crossCvrFeat_gender_age_cols + crossCvrFeat_occu_cols + crossCvrFeat_shop_cols
# drop_features += list(filter(lambda x: 'gender' not in x, crossCvr_cols))

features = train.drop(drop_features, axis=1).columns.tolist()

In [83]:
features

['context_page_id',
 'item_brand_id',
 'item_city_id',
 'item_collected_level',
 'item_id',
 'item_price_level',
 'item_pv_level',
 'item_sales_level',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_score_delivery',
 'shop_score_description',
 'shop_score_service',
 'shop_star_level',
 'user_age_level',
 'user_gender_id',
 'user_id',
 'user_occupation_id',
 'user_star_level',
 'hour',
 'minute',
 'hour_range',
 'acc_jac_sim',
 'acc_match_ratio_sim',
 'best_predict_cat_hit_level',
 'best_prop_jac',
 'best_prop_match_ratio',
 'item_cat_1',
 'item_cat_2',
 'predict_cat_hit',
 'shop_review_positive_num',
 'shop_item_sales_min',
 'cat_item_cnt',
 'shop_item_price_min',
 'shop_user_age_mean',
 'shop_cat_num',
 'buy_context_ts_user_id_hhour1_before',
 'dis_context_ts_user_id_hhour1_before',
 'buy_context_ts_user_id_hhour2_before',
 'dis_context_ts_user_id_hhour2_before',
 'buy_context_ts_user_id_hhour4_before',
 'dis_context_ts_user_id_hhour4_before',
 'dis_context

In [89]:
# 这里别忘了删除掉ratio！！！！！！！

In [90]:
features

['context_page_id',
 'item_brand_id',
 'item_city_id',
 'item_collected_level',
 'item_id',
 'item_price_level',
 'item_pv_level',
 'item_sales_level',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_score_delivery',
 'shop_score_description',
 'shop_score_service',
 'shop_star_level',
 'user_age_level',
 'user_gender_id',
 'user_id',
 'user_occupation_id',
 'user_star_level',
 'hour',
 'minute',
 'hour_range',
 'acc_jac_sim',
 'acc_match_ratio_sim',
 'best_predict_cat_hit_level',
 'best_prop_jac',
 'best_prop_match_ratio',
 'item_cat_1',
 'item_cat_2',
 'predict_cat_hit',
 'shop_review_positive_num',
 'shop_item_sales_min',
 'cat_item_cnt',
 'shop_item_price_min',
 'shop_user_age_mean',
 'shop_cat_num',
 'buy_context_ts_user_id_hhour1_before',
 'dis_context_ts_user_id_hhour1_before',
 'buy_context_ts_user_id_hhour2_before',
 'dis_context_ts_user_id_hhour2_before',
 'buy_context_ts_user_id_hhour4_before',
 'dis_context_ts_user_id_hhour4_before',
 'dis_context

# load data

- 现在只需运行以下代码

In [2]:
train, test = pd.read_pickle('train_all'), pd.read_pickle('test_all')

In [3]:
total_pd = train.append(test)

In [4]:
id_features_more = ["item_brand_id", "item_id", "shop_id", "user_id"]
for fea in id_features_more:
    # 去掉低频的id特征
    target = total_pd[fea]
    cats = target.value_counts()
    num = cats.describe()['75%']
    print(fea, num, cats.shape[0])
    index_val = cats[cats <= num].index

    target.loc[target.isin(index_val)] = -1
    total_pd[fea] = target

item_brand_id 108.0 11097
item_id 29.0 57493
shop_id 76.0 19659
user_id 3.0 1115301


In [5]:
train.shape,test.shape

((1077175, 247), (1729656, 247))

In [6]:
train = total_pd.iloc[:1077175]
test = total_pd.iloc[1077175:]

In [4]:
# index = pd.read_pickle('../glq/2.data/data_all')['index']
# time = pd.read_pickle('../glq/2.data/data_all')['context_timestamp:day']
# gc.collect()

21

In [5]:
# t_test = pd.DataFrame(pd.read_pickle('../glq/2.data/myfeature/proportion'))
# t_test['index'] = index
# t_test.sort_values('index',inplace = True)
# t_test = t_test.iloc[10432036:]
# t_test.drop(['index'],axis = 1,inplace =True)
# t_test.index = test.index
# gc.collect()

0

In [6]:
# t_train = pd.DataFrame(pd.read_pickle('../glq/2.data/myfeature/proportion'))
# t_train['index'] = index
# t_train['context_timestamp:day'] = time
# t_train.sort_values('index',inplace = True)
# t_train = t_train.iloc[:10432035]
# t_train = t_train[t_train['context_timestamp:day'] == 7]
# t_train.drop(['index','context_timestamp:day'],axis = 1,inplace =True)
# t_train.index = train.index
# gc.collect()

0

In [7]:
# def concat(total_pd, new_pd):
#     if type(new_pd) == pd.DataFrame:
#         for col in new_pd.columns:
#             total_pd[col] = new_pd[col]
#     elif type(new_pd) == pd.Series:
#         total_pd[new_pd.name] = new_pd
#     return total_pd

In [8]:
# print(train.shape,test.shape)
# train = concat(train,t_train)
# test = concat(test,t_test)
# gc.collect()
# print(train.shape,test.shape)

(1077175, 247) (1729656, 247)
(1077175, 250) (1729656, 250)


In [9]:
# del t_train,t_test
# gc.collect()

14

In [7]:
off_train, off_valid = train[train.hour < 10], train[train.hour >= 10]

In [8]:
off_train.shape, off_valid.shape, test.shape

((807438, 247), (269737, 247), (1729656, 247))

In [9]:
features = ['context_page_id',
 'item_brand_id',
 'item_city_id',
 'item_collected_level',
 'item_id',
 'item_price_level',
 'item_pv_level',
 'item_sales_level',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_score_delivery',
 'shop_score_description',
 'shop_score_service',
 'shop_star_level',
 'user_age_level',
 'user_gender_id',
 'user_id',
 'user_occupation_id',
 'user_star_level',
 'hour',
 'minute',
 'hour_range',
 'acc_jac_sim',
 'acc_match_ratio_sim',
 'best_predict_cat_hit_level',
 'best_prop_jac',
 'best_prop_match_ratio',
 'item_cat_1',
 'item_cat_2',
 'predict_cat_hit',
 'shop_review_positive_num',
 'shop_item_sales_min',
 'cat_item_cnt',
 'shop_item_price_min',
 'shop_user_age_mean',
 'shop_cat_num',
 'buy_context_ts_user_id_hhour1_before',
 'dis_context_ts_user_id_hhour1_before',
 'buy_context_ts_user_id_hhour2_before',
 'dis_context_ts_user_id_hhour2_before',
 'buy_context_ts_user_id_hhour4_before',
 'dis_context_ts_user_id_hhour4_before',
 'dis_context_ts_user_id_hhour1_after',
 'dis_context_ts_user_id_hhour2_after',
 'dis_context_ts_user_id_hhour4_after',
 'buy_context_ts_item_id_hhour1_before',
 'dis_context_ts_item_id_hhour1_before',
 'buy_context_ts_item_id_hhour2_before',
 'dis_context_ts_item_id_hhour2_before',
 'buy_context_ts_item_id_hhour4_before',
 'dis_context_ts_item_id_hhour4_before',
 'dis_context_ts_item_id_hhour1_after',
 'dis_context_ts_item_id_hhour2_after',
 'dis_context_ts_item_id_hhour4_after',
 'buy_context_ts_shop_id_hhour1_before',
 'dis_context_ts_shop_id_hhour1_before',
 'buy_context_ts_shop_id_hhour2_before',
 'dis_context_ts_shop_id_hhour2_before',
 'buy_context_ts_shop_id_hhour4_before',
 'dis_context_ts_shop_id_hhour4_before',
 'dis_context_ts_shop_id_hhour1_after',
 'dis_context_ts_shop_id_hhour2_after',
 'dis_context_ts_shop_id_hhour4_after',
 'buy_context_ts_user_id_item_id_hhour1_before',
 'dis_context_ts_user_id_item_id_hhour1_before',
 'buy_context_ts_user_id_item_id_hhour2_before',
 'dis_context_ts_user_id_item_id_hhour2_before',
 'buy_context_ts_user_id_item_id_hhour4_before',
 'dis_context_ts_user_id_item_id_hhour4_before',
 'dis_context_ts_user_id_item_id_hhour1_after',
 'dis_context_ts_user_id_item_id_hhour2_after',
 'dis_context_ts_user_id_item_id_hhour4_after',
 'buy_context_ts_user_id_shop_id_hhour1_before',
 'dis_context_ts_user_id_shop_id_hhour1_before',
 'buy_context_ts_user_id_shop_id_hhour2_before',
 'dis_context_ts_user_id_shop_id_hhour2_before',
 'buy_context_ts_user_id_shop_id_hhour4_before',
 'dis_context_ts_user_id_shop_id_hhour4_before',
 'dis_context_ts_user_id_shop_id_hhour1_after',
 'dis_context_ts_user_id_shop_id_hhour2_after',
 'dis_context_ts_user_id_shop_id_hhour4_after',
 'user_id_context_time_gap',
 'user_id_context_time_gap_next',
 'item_id_context_time_gap',
 'item_id_context_time_gap_next',
 'shop_id_context_time_gap',
 'shop_id_context_time_gap_next',
 'user_id_item_id_context_time_gap',
 'user_id_item_id_context_time_gap_next',
 'user_id_shop_id_context_time_gap',
 'item_id_HistoryCVR',
 'shop_id_HistoryCVR',
 'user_id_HistoryCVR',
 'user_age_level_HistoryCVR',
 'context_page_id_HistoryCVR',
 'item_brand_id_HistoryCVR',
 'item_price_level_HistoryCVR',
 'item_sales_level_HistoryCVR',
 'item_collected_level_HistoryCVR',
 'item_cat_1_HistoryCVR',
 'user_star_level_HistoryCVR',
 'shop_review_num_level_HistoryCVR',
 'shop_star_level_HistoryCVR',
 'hour_HistoryCVR',
 'hour_range_HistoryCVR',
 'his_item_cat_1_user_age_level_tradeRatio',
 'his_item_cat_1_user_gender_id_tradeRatio',
 'his_item_cat_1_user_occupation_id_tradeRatio',
 'his_shop_id_user_age_level_tradeRatio',
 'his_shop_id_user_gender_id_tradeRatio',
 'his_shop_id_user_occupation_id_tradeRatio',
 'his_item_price_level_user_age_level_tradeRatio',
 'his_item_price_level_user_gender_id_tradeRatio',
 'his_item_price_level_user_occupation_id_tradeRatio',
 'his_item_brand_id_item_id_tradeRatio',
 'his_item_cat_1_item_id_tradeRatio',
 'his_item_cat_1_shop_id_tradeRatio',
 'his_shop_id_item_id_tradeRatio',
 'initUserTotalClickCount',
 'initUserDayClickCount',
 'initUserHourClickCount',
 'initUserTotalItemClickCount',
 'initUserDayItemClickCount',
 'initUserTotalBrandClickCount',
 'initUserDayBrandClickCount',
 'initUserTotalCat1ClickCount',
 'initUserDayCat1ClickCount',
 'initUserHourCat1ClickCount',
 'initUserTotalCa2ClickCount',
 'initUserDayCat2ClickCount',
 'initUserTotalShopClickCount',
 'initUserDayShopClickCount',
 'initUserHourShopClickCount',
 'initUserTotalPriceClickCount',
 'initUserDayPriceClickCount',
 'initUserTotalCityClickCount',
 'initUserDayCityClickCount',
 'preDay_initItemClickCount',
 'preDay_initItemTradeCount',
 'preDay_initShopClickCount',
 'preDay_initShopTradeCount',
 'preDay_initBrandClickCount',
 'preDay_initBrandTradeCount',
 'preDay_initCat1ClickCount',
 'preDay_initCat1TradeCount',
 'preDay_initCat2ClickCount',
 'preDay_initCat2TradeCount',
 'userTodayAlreadyTradeCount',
 'userItemTodayAlreadyTradeCount',
 'userCat1TodayAlreadyTradeCount',
 'userShopTodayAlreadyTradeCount',
 'userBrandTodayAlreadyTradeCount',
 'userDayClickRank',
 'userDayItemClickRank',
 'userDayCatClickRank',
 'userDayShopClickRank',
 'userDayBrandClickRank',
 'item_price_level_subHorizonMean',
 'item_sales_level_subHorizonMean',
 'shop_review_num_level_subHorizonMean',
 'shop_score_service_subHorizonMean',
 'shop_score_delivery_subHorizonMean',
 'shop_score_description_subHorizonMean',
 'shop_review_positive_rate_subHorizonMean',

]

In [10]:
len(features)

165

In [ ]:
# del train
# gc.collect()

# LGB训练

In [11]:
off_train_all = off_train

In [95]:
off_train = off_train_all.sample(frac = 0.9,axis=0,random_state = 9999)

In [96]:
lgbParam = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 64,
    'learning_rate': 0.05,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'is_unbalance': False,
    'verbose': 0
}
# 线下验证
# lgb_train = lgb.Dataset(off_train[features], off_train['is_trade'])
# lgb_eval = lgb.Dataset(off_test[features], off_test['is_trade'], reference=lgb_train)
# lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=5000, early_stopping_rounds=15, valid_sets=lgb_eval)

lgb_train = lgb.Dataset(off_train[features], off_train['is_trade'])
lgb_eval = lgb.Dataset(off_valid[features], off_valid['is_trade'], reference=lgb_train)
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=2000, valid_sets=lgb_eval, early_stopping_rounds=50)

[1]	valid_0's binary_logloss: 0.652649
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's binary_logloss: 0.615991
[3]	valid_0's binary_logloss: 0.582686
[4]	valid_0's binary_logloss: 0.552344
[5]	valid_0's binary_logloss: 0.524593
[6]	valid_0's binary_logloss: 0.49913
[7]	valid_0's binary_logloss: 0.475731
[8]	valid_0's binary_logloss: 0.454158
[9]	valid_0's binary_logloss: 0.43425
[10]	valid_0's binary_logloss: 0.415874
[11]	valid_0's binary_logloss: 0.398862
[12]	valid_0's binary_logloss: 0.383085
[13]	valid_0's binary_logloss: 0.368458
[14]	valid_0's binary_logloss: 0.354878
[15]	valid_0's binary_logloss: 0.342269
[16]	valid_0's binary_logloss: 0.330535
[17]	valid_0's binary_logloss: 0.319627
[18]	valid_0's binary_logloss: 0.309458
[19]	valid_0's binary_logloss: 0.299993
[20]	valid_0's binary_logloss: 0.291166
[21]	valid_0's binary_logloss: 0.282912
[22]	valid_0's binary_logloss: 0.27522
[23]	valid_0's binary_logloss: 0.268029
[24]	valid_0's binary_logloss:

[203]	valid_0's binary_logloss: 0.162751
[204]	valid_0's binary_logloss: 0.162748
[205]	valid_0's binary_logloss: 0.162747
[206]	valid_0's binary_logloss: 0.162743
[207]	valid_0's binary_logloss: 0.162739
[208]	valid_0's binary_logloss: 0.162741
[209]	valid_0's binary_logloss: 0.162731
[210]	valid_0's binary_logloss: 0.162731
[211]	valid_0's binary_logloss: 0.162741
[212]	valid_0's binary_logloss: 0.162738
[213]	valid_0's binary_logloss: 0.162734
[214]	valid_0's binary_logloss: 0.162731
[215]	valid_0's binary_logloss: 0.162728
[216]	valid_0's binary_logloss: 0.162719
[217]	valid_0's binary_logloss: 0.162723
[218]	valid_0's binary_logloss: 0.16273
[219]	valid_0's binary_logloss: 0.162731
[220]	valid_0's binary_logloss: 0.16273
[221]	valid_0's binary_logloss: 0.16273
[222]	valid_0's binary_logloss: 0.162732
[223]	valid_0's binary_logloss: 0.162728
[224]	valid_0's binary_logloss: 0.162722
[225]	valid_0's binary_logloss: 0.162725
[226]	valid_0's binary_logloss: 0.162718
[227]	valid_0's bin

In [55]:
bst_now = lgbM.best_iteration
val1 = lgbM.predict(off_valid[features]) # 1111,0.1626334699309283
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test1 = lgbM.predict(test[features]) # 1111

In [62]:
bst_now = lgbM.best_iteration
val2 = lgbM.predict(off_valid[features]) # 2222,0.16238135755997427
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test2 = lgbM.predict(test[features]) # 2222

In [67]:
bst_now = lgbM.best_iteration
val3 = lgbM.predict(off_valid[features]) # 3333,0.16232054891066366
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test3 = lgbM.predict(test[features]) # 3333

In [72]:
bst_now = lgbM.best_iteration
val4 = lgbM.predict(off_valid[features]) # 4444, 0.1622630958278548
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test4 = lgbM.predict(test[features]) # 4444

In [77]:
bst_now = lgbM.best_iteration
val5 = lgbM.predict(off_valid[features]) # 5555,0.16220347239945873
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test5 = lgbM.predict(test[features]) # 5555

In [82]:
bst_now = lgbM.best_iteration
val6 = lgbM.predict(off_valid[features]) # 6666,0.1621927042859721
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test6 = lgbM.predict(test[features]) # 6666,

In [87]:
bst_now = lgbM.best_iteration
val7 = lgbM.predict(off_valid[features]) # 7777,0.16217402601979042
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test7 = lgbM.predict(test[features]) # 7777,

In [92]:
bst_now = lgbM.best_iteration
val8 = lgbM.predict(off_valid[features]) # 8888,0.162180544960351
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test8 = lgbM.predict(test[features]) # 8888,

In [97]:
bst_now = lgbM.best_iteration
val9 = lgbM.predict(off_valid[features]) # 9999,0.16216385039056
on_train = off_train.append(off_valid)

lgb_train = lgb.Dataset(on_train[features], on_train['is_trade'])
lgbM = lgb.train(lgbParam, lgb_train, num_boost_round=bst_now)
test9 = lgbM.predict(test[features]) # 9999,

In [98]:
val = (val1 + val2 + val3 + val4 + val5 + val6 + val7 + val8 + val9) / 9.0

In [99]:
log_loss(off_valid['is_trade'],val)

0.16216385039056

In [111]:
test_res = (test1 + test2 + test3 + test4 + test5 + test6) / 6.0

In [107]:
pd.Series(test1).to_pickle('test1')
pd.Series(test2).to_pickle('test2')
pd.Series(test3).to_pickle('test3')
pd.Series(test4).to_pickle('test4')
pd.Series(test5).to_pickle('test5')
pd.Series(test6).to_pickle('test6')

In [160]:
lgbFeatureScore = pd.DataFrame({'feature': lgbM.feature_name(), 'score': lgbM.feature_importance()}).sort_values(by='score', ascending=False)
print(lgbFeatureScore.values)

[['user_id_context_time_gap_next' 790]
 ['item_id_HistoryCVR' 572]
 ['item_sales_level' 553]
 ['shop_id_HistoryCVR' 430]
 ['acc_jac_sim' 403]
 ['initUserDayCat1ClickCount' 383]
 ['user_id_item_id_context_time_gap' 379]
 ['acc_match_ratio_sim' 378]
 ['item_id_context_time_gap' 373]
 ['user_id' 366]
 ['user_id_context_time_gap' 344]
 ['shop_user_age_mean' 340]
 ['item_brand_id_HistoryCVR' 339]
 ['user_id_HistoryCVR' 330]
 ['userTodayAlreadyTradeCount' 321]
 ['item_id_context_time_gap_next' 318]
 ['user_star_level' 317]
 ['item_sales_level_subHorizonMean' 291]
 ['minute' 290]
 ['initUserTotalClickCount' 289]
 ['his_shop_id_user_age_level_tradeRatio' 275]
 ['shop_id_context_time_gap_next' 275]
 ['shop_review_num_level_subHorizonMean' 268]
 ['his_shop_id_user_occupation_id_tradeRatio' 260]
 ['item_id' 258]
 ['his_shop_id_user_gender_id_tradeRatio' 258]
 ['shop_id_context_time_gap' 233]
 ['shop_score_description' 233]
 ['item_cat_1_HistoryCVR' 220]
 ['item_brand_id' 218]
 ['user_id_item_id_c

In [112]:
test_b = pd.read_table('../glq/2.data/round2_test_b.txt', delim_whitespace=True, index_col=None)

In [113]:
testB_len = len(test_b)
print(testB_len)
test.loc[:, 'predicted_score'] = test_res
res = test[['instance_id', 'predicted_score']].iloc[-1*testB_len:].reset_index(drop=True)
print(len(res))
print(res.isnull().any())
res.to_csv('res-lgb-subsample-final.txt', index=False, sep=' ', line_terminator='\r')  # 保存在线提交结果

1209768
1209768
instance_id        False
predicted_score    False
dtype: bool


In [117]:
res['predicted_score'].std()

0.03515847418696045

In [27]:
test.loc[:, 'predicted_score'] = lgbM.predict(test[features])
res = test[['instance_id', 'predicted_score']]
print(res.isnull().any())
res.to_csv('result/lgb-0429-150round.txt', index=False, sep=' ', line_terminator='\r')  # 保存在线提交结果

instance_id        False
predicted_score    False
dtype: bool


# XGB训练

In [16]:
xgbParam = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'eval_metric': 'logloss',
    'seed': 3333,
    'gamma': 0, # 节点分裂所需的最小损失函数下降值
    'scale_pos_weight': 1,
    # 'missing': -999,
    'alpha': 0.1,
    'min_child_weight': 5,
    'silent': 1,
#     'base_score': 0.0188
}
xgbtrain = xgb.DMatrix(off_train[features], off_train['is_trade'])
xgbvalid = xgb.DMatrix(off_valid[features], off_valid['is_trade'])
watchlist = [(xgbtrain, 'train'), (xgbvalid, 'valid')]
xgbModel = xgb.train(xgbParam, xgbtrain, evals=watchlist, num_boost_round=1500, early_stopping_rounds=50)

[0]	train-logloss:0.652882	valid-logloss:0.652649
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[1]	train-logloss:0.616503	valid-logloss:0.616007
[2]	train-logloss:0.583392	valid-logloss:0.582686
[3]	train-logloss:0.553262	valid-logloss:0.552339
[4]	train-logloss:0.525713	valid-logloss:0.524569
[5]	train-logloss:0.500475	valid-logloss:0.499155
[6]	train-logloss:0.477257	valid-logloss:0.47575
[7]	train-logloss:0.455925	valid-logloss:0.454184
[8]	train-logloss:0.436246	valid-logloss:0.434341
[9]	train-logloss:0.418069	valid-logloss:0.415972
[10]	train-logloss:0.401256	valid-logloss:0.398988
[11]	train-logloss:0.385671	valid-logloss:0.383232
[12]	train-logloss:0.371239	valid-logloss:0.368616
[13]	train-logloss:0.357843	valid-logloss:0.355045
[14]	train-logloss:0.345412	valid-logloss:0.342452
[15]	train-logloss:0.333826	valid-logloss:0.330734
[16]	train-logloss:0.323055	valid-logloss:0.

[158]	train-logloss:0.169026	valid-logloss:0.16322
[159]	train-logloss:0.168992	valid-logloss:0.163209
[160]	train-logloss:0.168955	valid-logloss:0.163196
[161]	train-logloss:0.168899	valid-logloss:0.163172
[162]	train-logloss:0.168863	valid-logloss:0.163152
[163]	train-logloss:0.168836	valid-logloss:0.163138
[164]	train-logloss:0.168783	valid-logloss:0.163116
[165]	train-logloss:0.168758	valid-logloss:0.163105
[166]	train-logloss:0.168702	valid-logloss:0.163085
[167]	train-logloss:0.16866	valid-logloss:0.163075
[168]	train-logloss:0.168631	valid-logloss:0.163064
[169]	train-logloss:0.168604	valid-logloss:0.163053
[170]	train-logloss:0.168574	valid-logloss:0.16304
[171]	train-logloss:0.168539	valid-logloss:0.163032
[172]	train-logloss:0.168489	valid-logloss:0.163015
[173]	train-logloss:0.168449	valid-logloss:0.162999
[174]	train-logloss:0.168418	valid-logloss:0.162989
[175]	train-logloss:0.168383	valid-logloss:0.162984
[176]	train-logloss:0.168336	valid-logloss:0.16297
[177]	train-logl

[317]	train-logloss:0.164287	valid-logloss:0.16223
[318]	train-logloss:0.16427	valid-logloss:0.162234
[319]	train-logloss:0.164243	valid-logloss:0.162237
[320]	train-logloss:0.164223	valid-logloss:0.162237
[321]	train-logloss:0.164209	valid-logloss:0.162234
[322]	train-logloss:0.164185	valid-logloss:0.162235
[323]	train-logloss:0.164166	valid-logloss:0.162234
[324]	train-logloss:0.164147	valid-logloss:0.162229
[325]	train-logloss:0.164121	valid-logloss:0.162228
[326]	train-logloss:0.164104	valid-logloss:0.162228
[327]	train-logloss:0.164082	valid-logloss:0.162226
[328]	train-logloss:0.164063	valid-logloss:0.162225
[329]	train-logloss:0.164051	valid-logloss:0.162224
[330]	train-logloss:0.164026	valid-logloss:0.162219
[331]	train-logloss:0.163993	valid-logloss:0.162217
[332]	train-logloss:0.163976	valid-logloss:0.162214
[333]	train-logloss:0.163952	valid-logloss:0.162211
[334]	train-logloss:0.163947	valid-logloss:0.16221
[335]	train-logloss:0.163918	valid-logloss:0.162211
[336]	train-log

In [17]:
bst_iter = xgbModel.best_iteration
xgbtrain = xgb.DMatrix(train[features], train['is_trade'])
watchlist = [(xgbtrain, 'train')]
_xgbModel = xgb.train(xgbParam, xgbtrain, evals=watchlist, num_boost_round=bst_iter)

[0]	train-logloss:0.652755
[1]	train-logloss:0.616251
[2]	train-logloss:0.583054
[3]	train-logloss:0.552797
[4]	train-logloss:0.525136
[5]	train-logloss:0.499818
[6]	train-logloss:0.476539
[7]	train-logloss:0.455107
[8]	train-logloss:0.435342
[9]	train-logloss:0.417105
[10]	train-logloss:0.400216
[11]	train-logloss:0.38459
[12]	train-logloss:0.370107
[13]	train-logloss:0.356652
[14]	train-logloss:0.344189
[15]	train-logloss:0.332569
[16]	train-logloss:0.321744
[17]	train-logloss:0.311709
[18]	train-logloss:0.30233
[19]	train-logloss:0.293598
[20]	train-logloss:0.28547
[21]	train-logloss:0.277898
[22]	train-logloss:0.2708
[23]	train-logloss:0.264185
[24]	train-logloss:0.258008
[25]	train-logloss:0.252247
[26]	train-logloss:0.246884
[27]	train-logloss:0.241865
[28]	train-logloss:0.237182
[29]	train-logloss:0.232802
[30]	train-logloss:0.228707
[31]	train-logloss:0.224889
[32]	train-logloss:0.221317
[33]	train-logloss:0.217974
[34]	train-logloss:0.214855
[35]	train-logloss:0.211938
[36]	tr

[288]	train-logloss:0.163956
[289]	train-logloss:0.163942
[290]	train-logloss:0.163912
[291]	train-logloss:0.163892
[292]	train-logloss:0.16387
[293]	train-logloss:0.163847
[294]	train-logloss:0.163829
[295]	train-logloss:0.163811
[296]	train-logloss:0.163791
[297]	train-logloss:0.163779
[298]	train-logloss:0.163753
[299]	train-logloss:0.163726
[300]	train-logloss:0.163711
[301]	train-logloss:0.163691
[302]	train-logloss:0.163664
[303]	train-logloss:0.163637
[304]	train-logloss:0.163626
[305]	train-logloss:0.163609
[306]	train-logloss:0.163587
[307]	train-logloss:0.163567
[308]	train-logloss:0.163546
[309]	train-logloss:0.163534
[310]	train-logloss:0.16351
[311]	train-logloss:0.163481
[312]	train-logloss:0.163452
[313]	train-logloss:0.163426
[314]	train-logloss:0.163404
[315]	train-logloss:0.163395
[316]	train-logloss:0.163385
[317]	train-logloss:0.163374
[318]	train-logloss:0.163361
[319]	train-logloss:0.163336
[320]	train-logloss:0.163313
[321]	train-logloss:0.163293
[322]	train-logl

In [18]:
test_b = pd.read_table('../glq/2.data/round2_test_b.txt', delim_whitespace=True, index_col=None)

In [19]:
testB_len = len(test_b)
print(testB_len)
xgbtest = xgb.DMatrix(test.iloc[-testB_len:][features])
test_b.loc[:, 'predicted_score'] = _xgbModel.predict(xgbtest)
res = test_b[['instance_id', 'predicted_score']].reset_index(drop=True)
print(len(res))
print(res.isnull().any())
res.to_csv('res-xgb-0515-3.txt', index=False, sep=' ', line_terminator='\r')  # 保存在线提交结果

1209768
1209768
instance_id        False
predicted_score    False
dtype: bool


In [175]:
res['predicted_score'].mean()

0.04031452536582947

In [176]:
# 用来修正
test_b[['user_id', 'predicted_score', 'instance_id']].reset_index(drop=True).to_csv('test-res-0514.csv', index=False)  # 保存在线提交结果

# Catboost训练
- 参考：https://blog.csdn.net/AiirrrrYee/article/details/78224232

In [ ]:
from catboost import CatBoostClassifier
from catboost import Pool

In [ ]:
model = CatBoostClassifier(
    fold_permutation_block_size=None, # 在数据随机排列前分块，值越小越慢。默认1。
    nan_mode=None, # 处理输入数据中缺失值的方法，包括Forbidden(禁止存在缺失)，Min(用最小值补)，Max(用最大值补)。默认Min。
    ignored_features=None,
    use_best_model=None,
    custom_loss=None,
    loss_function='Logloss', # 支持的有RMSE, Logloss, MAE, CrossEntropy, Quantile, LogLinQuantile, Multiclass, MultiClassOneVsAll, MAPE, Poisson。默认Logloss。
    eval_metric=None,
    iterations=None, # 最大树数。默认500。
    depth=None, #  树深，最大16，建议在1到10之间。默认6。
    learning_rate=None, # 学习率。默认0.03。
    rsm=None,
    leaf_estimation_method=None,
    l2_leaf_reg=None,  # l2正则参数。默认3
    has_time=None,
    name=None,
    fold_len_multiplier=None,
    approx_on_full_history=None,
    classes_count=None,
    class_weights=None,
    one_hot_max_size=None,
    random_strength=None,
    bagging_temperature=None,
    
    # Binarization settings 
    border_count=None, # numerical features的分割数，范围1－255。默认128。 
    feature_border_type=None, # numerical features的二值化模式，Median, Uniform, UniformAndQuantiles, MaxSumLog, MinEntropy, GreedyLogSum。默认MiniEntropy。
    # Overfitting detection settings 
    od_pval=None, # 使用IncToDec时的阈值，值越大越早检测出过拟合。默认0（不使用过拟合检测）。
    od_wait=None, # 在最小化损失函数后的迭代次数。使用InctoDec时，表示当达到阈值后，忽略过拟合检测，继续训练。使用Iter时，表示达到指定次数后，停止训练。默认20。
    od_type=None, # 过拟合检测类型：IncToDec/ Iter。默认IncToDec。 
    # CTR settings 
    counter_calc_method=None, # 计算点击率类型的方法，PrefixTest考虑测试集中当前对象，FullTest考虑测试集中所有对象，SkipTest不考虑测试集中的对象，Full考虑训练和测试集中的全部对象。默认None（PrefixTest）。 
    ctr_leaf_count_limit=None,
    store_all_simple_ctr=None,
    max_ctr_complexity=None,
    ctr_description=None,
    # Performance settings
    thread_count=None,
    # Output settings 
    verbose=None,
    train_dir=None,
    allow_writing_files=None,
    save_snapshot=None,
    snapshot_file=None,
    logging_level=None,
    
    
    
    leaf_estimation_iterations=None,
    model_size_reg=None,
    random_seed=None,
    metric_period=None,
    custom_metric=None,
    used_ram_limit=None,
    gpu_ram_part=None,
    boosting_type=None,
    simple_ctr=None,                         
    combinations_ctr=None,
    per_feature_ctr=None,
    task_type=None,
    device_config=None,
    devices=None,
    bootstrap_type=None,
    subsample=None,
    max_depth=None,
    n_estimators=None,
    num_boost_round=None,
    num_trees=None,
    colsample_bylevel=None,
    random_state=None,
    reg_lambda=None,
    objective=None,
    eta=None,
    max_bin=None,
    scale_pos_weight=None,
    gpu_cat_features_storage=None)

model.fit(train_data, train_label, cat_features=[0,2,5])
